In [9]:
import sys
IN_COLAB = "google.colab" in sys.modules
def allow_notebook_for_test():
    print("Testing the totalVI notebook")
show_plot = True
test_mode = False
save_path = "data/"
if not test_mode:
    save_path = "../../data"
if IN_COLAB:
    !pip install --quiet git+https://github.com/yoseflab/scvi@stable#egg=scvi[notebooks]
import scanpy as sc
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import anndata
import os
from scvi.dataset import GeneExpressionDataset, DownloadableAnnDataset
from scvi.models import TOTALVI
from scvi.inference import TotalPosterior, TotalTrainer
from scvi import set_seed
if IN_COLAB:
    %matplotlib inline
# Sets the random seed for torch and numpy
set_seed(0)
sc.set_figure_params(figsize=(4, 4))
dataset1 = DownloadableAnnDataset(
    "pbmc_10k_protein.h5ad",
    save_path=save_path,
    url="https://github.com/YosefLab/scVI-data/raw/master/pbmc_10k_protein_v3.h5ad?raw=true",
    cell_measurements_col_mappings={"protein_expression":"protein_names"},
)
dataset2 = DownloadableAnnDataset(
    "pbmc_5k_protein.h5ad",
    save_path=save_path,
    url="https://github.com/YosefLab/scVI-data/raw/master/pbmc_5k_protein_v3.h5ad?raw=true",
    cell_measurements_col_mappings={"protein_expression":"protein_names"},
)
dataset = GeneExpressionDataset()
dataset.populate_from_datasets([dataset1, dataset2])
dataset.subsample_genes(4000)
use_cuda = True
lr = 4e-3

n_epochs = 100

import time
times = []
for i in range(5):
    totalvae = TOTALVI(
        dataset.nb_genes,
        len(dataset.protein_names),
        n_batch=dataset.n_batches,
    )
    trainer = TotalTrainer(
        totalvae,
        dataset,
        train_size=0.90,
        test_size=0.10,
        use_cuda=use_cuda,
        frequency=1,
        batch_size=256,
        early_stopping_kwargs="auto" if not test_mode else None
    )
    start = time.time()
    trainer.train(n_epochs=n_epochs)
    end = time.time()
    print(end-start)
    times.append(end-start)

[2020-08-06 21:34:56,988] INFO - scvi.dataset.dataset | File /data1/galen/data/pbmc_10k_protein.h5ad already downloaded
[2020-08-06 21:34:59,643] INFO - scvi.dataset.dataset | Remapping batch_indices to [0,N]
[2020-08-06 21:34:59,646] INFO - scvi.dataset.dataset | Remapping labels to [0,N]
[2020-08-06 21:35:01,556] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2020-08-06 21:35:01,824] INFO - scvi.dataset.dataset | Downsampled from 6855 to 6855 cells
[2020-08-06 21:35:01,829] INFO - scvi.dataset.dataset | File /data1/galen/data/pbmc_5k_protein.h5ad already downloaded
[2020-08-06 21:35:03,542] INFO - scvi.dataset.dataset | Remapping batch_indices to [0,N]
[2020-08-06 21:35:03,543] INFO - scvi.dataset.dataset | Remapping labels to [0,N]
[2020-08-06 21:35:04,713] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2020-08-06 21:35:04,869] INFO - scvi.dataset.dataset | Downsampled from 3994 to 3994 cells
[2020-08-06 21:35:04,873] INFO - sc

/data1/galen/.pyenv/versions/miniconda3-latest/envs/scvi/lib/python3.8/site-packages/anndata/_core/anndata.py:119: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/data1/galen/.pyenv/versions/miniconda3-latest/envs/scvi/lib/python3.8/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):


[2020-08-06 21:35:18,131] INFO - scvi.dataset.dataset | Downsampling from 15792 to 4000 genes


/data1/galen/scVI/scvi/dataset/dataset.py:1850: FutureWarning: Slicing a positional slice with .loc is not supported, and will raise TypeError in a future version.  Use .loc with labels or .iloc with positions instead.
  df.loc[:n_top_genes, "highly_variable"] = True


[2020-08-06 21:35:19,035] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2020-08-06 21:35:19,258] INFO - scvi.dataset.dataset | Filtering non-expressing cells.
[2020-08-06 21:35:20,121] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2020-08-06 21:35:20,228] INFO - scvi.dataset.dataset | Downsampled from 10849 to 10849 cells
[2020-08-06 21:35:20,965] INFO - scvi.inference.inference | KL warmup phase may exceed overall training phase.If your applications rely on posterior quality, consider training for more epochs or reducing the kl warmup.
[2020-08-06 21:35:20,966] INFO - scvi.inference.inference | KL warmup for 8136.75 iterations
training: 100%|██████████| 100/100 [03:29<00:00,  2.10s/it]
[2020-08-06 21:38:51,524] INFO - scvi.inference.inference | Training is still in warming up phase. If your applications rely on the posterior quality, consider training for more epochs or reducing the kl warmup.
211.21467399597168
[2020-08-06 21:

In [10]:
times

[211.21467399597168,
 211.5851526260376,
 211.64002561569214,
 211.40065717697144,
 211.38593316078186]

In [11]:
n_epochs

100

In [12]:
np.save('master_totalvi_{}_epochs.npy'.format(n_epochs), times)

In [17]:
np.load('refactor_totalvi_200_epochs.npy')

array([443.58983016, 442.79905343, 443.70387053, 443.61185789,
       440.82842875, 444.72131658, 442.34703469, 444.42763734,
       442.47483492, 443.05410671, 442.24437475, 442.08427954,
       443.82903051, 443.97489548, 444.88595629])

In [19]:
np.load('master_totalvi_100_epochs.npy')

array([211.214674  , 211.58515263, 211.64002562, 211.40065718,
       211.38593316])

In [1]:
import sys
IN_COLAB = "google.colab" in sys.modules
def allow_notebook_for_test():
    print("Testing the basic tutorial notebook")
show_plot = True
test_mode = False
save_path = "data/"
if not test_mode:
    save_path = "../../data"
if IN_COLAB:
    !pip install --quiet git+https://github.com/yoseflab/scvi@stable#egg=scvi[notebooks]
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scvi.dataset import CortexDataset, RetinaDataset
from scvi.models import VAE
from scvi.inference import UnsupervisedTrainer, load_posterior
from scvi import set_seed
import torch
# Control UMAP numba warnings
import warnings; warnings.simplefilter('ignore')
if IN_COLAB:
    %matplotlib inline
# Sets torch and numpy random seeds, run after all scvi imports
set_seed(0)
gene_dataset = CortexDataset(save_path=save_path, total_genes=None)
gene_dataset.subsample_genes(1000)
# we make the gene names lower case just for this tutorial
# scVI datasets preserve the case of the gene names as they are in the source files
gene_dataset.make_gene_names_lower()
lr = 1e-3
use_cuda = False
vae = VAE(gene_dataset.nb_genes)
trainer = UnsupervisedTrainer(
    vae,
    gene_dataset,
    train_size=0.90,
    use_cuda=use_cuda,
    frequency=5,
)
import time
n_epochs = 1


[2020-08-11 19:03:01,905] INFO - scvi.dataset.dataset | File /data1/galen/data/expression.bin already downloaded
[2020-08-11 19:03:01,907] INFO - scvi.dataset.cortex | Loading Cortex data
[2020-08-11 19:03:18,843] INFO - scvi.dataset.cortex | Finished preprocessing Cortex data
[2020-08-11 19:03:19,335] INFO - scvi.dataset.dataset | Remapping batch_indices to [0,N]
[2020-08-11 19:03:19,337] INFO - scvi.dataset.dataset | Remapping labels to [0,N]
[2020-08-11 19:03:19,562] INFO - scvi.dataset.dataset | extracting highly variable genes using seurat_v3 flavor
[2020-08-11 19:03:23,169] INFO - scvi.dataset.dataset | Downsampling from 19972 to 1000 genes
[2020-08-11 19:03:23,232] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2020-08-11 19:03:23,252] INFO - scvi.dataset.dataset | Filtering non-expressing cells.
[2020-08-11 19:03:23,313] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2020-08-11 19:03:23,320] INFO - scvi.dataset.dataset | D

In [2]:
from scipy.sparse import csr_matrix
gene_dataset.X = csr_matrix(gene_dataset.X)

[2020-08-11 19:03:23,439] INFO - scvi.dataset.dataset | Computing the library size for the new data


In [3]:
gene_dataset.X

<3005x1000 sparse matrix of type '<class 'numpy.float32'>'
	with 450124 stored elements in Compressed Sparse Row format>

In [4]:
for n_epochs in [50,100,200,400]:
    times = []
    for i in range(5):
        vae = VAE(gene_dataset.nb_genes)
        trainer = UnsupervisedTrainer(
            vae,
            gene_dataset,
            train_size=0.90,
            use_cuda=use_cuda,
            frequency=5,
        )
        start = time.time()
        trainer.train(n_epochs=n_epochs)
        end = time.time()
        print(end-start)
        times.append(end-start)
    np.save('A_master_scvi_{}_epochs_cpu_sparse.npy'.format(n_epochs), times)    

[2020-08-11 19:03:35,978] INFO - scvi.inference.inference | KL warmup phase exceeds overall training phaseIf your applications rely on the posterior quality, consider training for more epochs or reducing the kl warmup.
[2020-08-11 19:03:35,979] INFO - scvi.inference.inference | KL warmup for 400 epochs
training: 100%|██████████| 50/50 [00:31<00:00,  1.58it/s]
[2020-08-11 19:04:07,533] INFO - scvi.inference.inference | Training is still in warming up phase. If your applications rely on the posterior quality, consider training for more epochs or reducing the kl warmup.
31.855679750442505
[2020-08-11 19:04:07,817] INFO - scvi.inference.inference | KL warmup phase exceeds overall training phaseIf your applications rely on the posterior quality, consider training for more epochs or reducing the kl warmup.
[2020-08-11 19:04:07,818] INFO - scvi.inference.inference | KL warmup for 400 epochs
training: 100%|██████████| 50/50 [00:31<00:00,  1.60it/s]
[2020-08-11 19:04:39,120] INFO - scvi.inferen

In [ ]:
    np.save('A_refactor_scvi_{}_epochs_cpu_X.npy'.format(n_epochs), times)